In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# check shape of the data

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")

print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")
# visualize single data instances

img_no = 0 #change the number to display other examples

first_number = x_train[img_no]
plt.imshow(first_number, cmap='gray') # visualize the numbers in gray mode
plt.show()
print(f"correct number: {y_train[img_no]}")
# reshaping the data
# reshaping pixels in a 28x28px image with greyscale, canal = 1. This is needed for the Keras API
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

# normalizing the data
# each pixel has a value between 0-255. Here we divide by 255, to get values from 0-1
x_train = x_train / 255
x_test = x_test / 255

print(f"train X shape: {x_train.shape}")
print(f"test X shape: {x_test.shape}")
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28,28,1)))
model.add(keras.layers.MaxPool2D(2, 2))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPool2D(2, 2))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPool2D(2, 2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))

model.add(keras.layers.Dense(32, activation='relu'))

model.add(keras.layers.Dense(10, activation='softmax')) #output are 10 classes, numbers from 0-9

#show model summary - how it looks
model.summary()
#compile the model - we want to have a multiple outcome
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])
#fit the model and return the history while training
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=1
)

# Part2
from minio import Minio
import os
# 检查并创建目录 (exist_ok=True 表示如果文件夹已存在则不报错)
save_path = "models/detect-digits.keras"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
keras.models.save_model(model, save_path)

import minio
import glob
from minio import Minio

# 1. 初始化 MinIO 客户端 (保持你的配置)
minio_client = Minio(
    "minio-service.kubeflow:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False
)
minio_bucket = "mlpipeline"

def smart_upload_to_minio(local_path, bucket_name, minio_base_path):
    """
    智能上传函数：自动识别文件或文件夹并上传到 MinIO
    """
    if not os.path.exists(local_path):
        print(f"错误: 本地路径 {local_path} 不存在")
        return

    if os.path.isfile(local_path):
        # 如果是单个文件 (针对 Keras 3 的 .keras 文件)
        file_name = os.path.basename(local_path)
        remote_path = os.path.join(minio_base_path, file_name).replace(os.sep, '/')
        minio_client.fput_object(bucket_name, remote_path, local_path)
        print(f"成功上传文件: {local_path} -> {remote_path}")

    elif os.path.isdir(local_path):
        # 如果是文件夹 (针对旧版 SavedModel 格式)
        for root, dirs, files in os.walk(local_path):
            for file in files:
                local_file = os.path.join(root, file)
                # 计算相对路径，保持文件夹结构
                rel_path = os.path.relpath(local_file, local_path)
                remote_path = os.path.join(minio_base_path, rel_path).replace(os.sep, '/')

                minio_client.fput_object(bucket_name, remote_path, local_file)
                print(f"成功上传文件夹内文件: {local_file} -> {remote_path}")


smart_upload_to_minio(
    local_path="models/detect-digits.keras",
    bucket_name=minio_bucket,
    minio_base_path="models"  # 在 MinIO 里存放在 models 目录下
)
# 最终路径 - s3://mlpipeline/models/detect-digits.keras


# Part3
#load model if already trained
model = keras.models.load_model("models/detect-digits.keras")
#keras.models.save_model(model,"detect-digits.keras")
# Test the model against the test dataset
# Returns the loss value & metrics values for the model in test mode.
model_loss, model_accuracy = model.evaluate(x=x_test,y=y_test)
# Confusion Matrix

# Generates output predictions for the input samples.
test_predictions = model.predict(x=x_test)

# Returns the indices of the maximum values along an axis.
test_predictions = np.argmax(test_predictions,axis=1) # the prediction outputs 10 values, we take the index number of the highest value, which is the prediction of the model

# generate confusion matrix
confusion_matrix = tf.math.confusion_matrix(labels=y_test,predictions=test_predictions)

# plot confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt
h = sns.heatmap(confusion_matrix,annot=True,fmt='d')
h.set(xlabel='Predicted', ylabel='Actual')
